In [1]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.utils import class_weight
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
import tensorflow as tf


IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 50

In [3]:
train_dir = 'DATA/melanoma_cancer_dataset/train'
test_dir = 'DATA/melanoma_cancer_dataset/test'

In [4]:
# Data Augmentation + Validation Split
datagen_train = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = datagen_train.flow_from_directory(
    train_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_generator = datagen_train.flow_from_directory(
    train_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

Found 7684 images belonging to 2 classes.
Found 1921 images belonging to 2 classes.


In [5]:
# Test Data
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMAGE_SIZE,
    batch_size=1,
    class_mode='categorical',
    shuffle=False
)

Found 1000 images belonging to 2 classes.


In [6]:
# Handle Class Imbalance
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights = dict(enumerate(class_weights))
print("Class Weights:", class_weights)

Class Weights: {0: 0.9605, 1: 1.0428881650380022}


In [7]:
# Model: ResNet50 + Custom Head
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(2, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy', tf.keras.metrics.AUC()])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [8]:
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


In [ ]:
# Training
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    class_weight=class_weights,
    callbacks=[early_stop]
)

Epoch 1/50
241/241 [==============================] - 106s 422ms/step - loss: 0.7335 - accuracy: 0.5131 - auc: 0.5227 - val_loss: 0.6817 - val_accuracy: 0.5128 - val_auc: 0.6353
Epoch 2/50
241/241 [==============================] - 106s 441ms/step - loss: 0.6771 - accuracy: 0.5747 - auc: 0.6056 - val_loss: 0.6697 - val_accuracy: 0.6903 - val_auc: 0.7891
Epoch 3/50
241/241 [==============================] - 110s 458ms/step - loss: 0.6662 - accuracy: 0.5997 - auc: 0.6435 - val_loss: 0.6630 - val_accuracy: 0.7720 - val_auc: 0.8136
Epoch 4/50
241/241 [==============================] - 111s 459ms/step - loss: 0.6593 - accuracy: 0.6240 - auc: 0.6698 - val_loss: 0.6561 - val_accuracy: 0.7569 - val_auc: 0.8065
Epoch 5/50
241/241 [==============================] - 119s 496ms/step - loss: 0.6524 - accuracy: 0.6227 - auc: 0.6757 - val_loss: 0.6435 - val_accuracy: 0.7439 - val_auc: 0.8482
Epoch 6/50
241/241 [==============================] - 108s 446ms/step - loss: 0.6437 - accuracy: 0.6372 - auc:

In [ ]:
# Plot Training History
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.legend()
plt.title('Training History')
plt.show()

In [ ]:
# Evaluate Model
y_true = test_generator.classes
y_pred_probs = model.predict(test_generator)
y_pred = np.argmax(y_pred_probs, axis=1)

In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# Classification Report
target_names = list(test_generator.class_indices.keys())
print(classification_report(y_true, y_pred, target_names=target_names))

In [ ]:
# AUC Score
y_true_onehot = tf.keras.utils.to_categorical(y_true, num_classes=2)
auc = roc_auc_score(y_true_onehot, y_pred_probs)
print("AUC Score:", auc)